In [49]:
import pickle
import pandas as pd
pd.set_option('display.max_columns',None)
import itertools
from functools import partial
from collections import defaultdict

import numpy as np
from dyntapy import get_toy_network
from dyntapy.visualization import show_network, show_demand
from dyntapy.supply_data import relabel_graph
from dyntapy.demand_data import od_graph_from_matrix, add_centroids
from dyntapy.results import get_od_flows
from dyntapy.assignments import StaticAssignment

import geopandas as gpd
from typing import Set, Tuple, List, Dict, Optional
import igraph
import random
import logging
from pathlib import Path
from pickle import load
import time
import networkx as nx
from zipfile import ZipFile
import os

# requires local installation of ireiat package using 
from ireiat.util.cacheable import CACHE_PATH

## Set up a toy problem from scratch

In [103]:
# make a graph
g = nx.DiGraph()

# (node_id, x, y)
nodes = [(1,2,np.sqrt(2)),
         (2,3.414213562373095,2.8284271247461903),
         (3,3.414213562373095,0),
         (4,4.82842712474619,np.sqrt(2))]
for n,x,y in nodes:
    g.add_node(n,x_coord=x,y_coord=y)

# (u,v, length, capacity, free_speed)
edges  = [(1,2,2,2000,80),
          (2,4,2,2000,80),
          (1,3,2,2000,80),
          (3,4,2,500,120)]

for u,v,length,capacity,free_speed in edges:
    g.add_edge(u,v,length=length,capacity=capacity,free_speed=free_speed,lanes=1)
    g.add_edge(v,u,length=length,capacity=capacity,free_speed=free_speed,lanes=1)

# make centroids
centroid_nodes = [(5,1,1),
                   (6,7,1)]
delta = .1
turn_nodes = [(7,1,1+delta),
              (8,7,1+delta)]

for n,x,y in centroid_nodes:
    g.add_node(n,x_coord=x,y_coord=y,centroid=True)
for n,x,y in turn_nodes:
    g.add_node(n,x_coord=x,y_coord=y)

# (u,v, length, capacity, free_speed)
CONNECTOR_CAPACITY = 100000
CONNECTOR_FREE_SPEED = 1000
CONNECTOR_LENGTH = .001
s = [CONNECTOR_LENGTH,CONNECTOR_CAPACITY,CONNECTOR_FREE_SPEED]
connector_edges = [(5,7,*s),
                   (7,1,*s),
                   (6,8,*s),
                   (8,4,*s)]

# straigh from centroid to graph
# connector_edges = [(5,1,*s),
#                    (6,4,*s)]

for u,v,length,capacity,free_speed in connector_edges:
    g.add_edge(u,v,length=length,capacity=capacity,free_speed=free_speed,connector=True,lanes=1)
    g.add_edge(v,u,length=length,capacity=capacity,free_speed=free_speed,connector=True,lanes=1)

labeled_g = relabel_graph(g)
show_network(labeled_g,euclidean=True)

In [99]:
# build the OD graph
od_graph = nx.DiGraph()
for idx, (n,x,y) in enumerate(centroid_nodes):
    od_graph.add_node(idx,x_coord=x,y_coord=y)
od_graph.add_edge(0,1,flow=3000)

In [100]:
%%time
sa = StaticAssignment(labeled_g, od_graph)

init passed successfully
CPU times: total: 0 ns
Wall time: 2.03 ms


In [101]:
result = sa.run(method='dial_b')

solution found, Dial B in iteration 1


In [102]:
result

StaticResult(link_costs=array([1.00000012e-06, 9.99999997e-07, 3.02287889e-02, 2.51094650e-02,
       9.99999997e-07, 2.50000004e-02, 3.02287889e-02, 2.50000004e-02,
       3.53486378e-02, 2.50000004e-02, 1.66666675e-02, 1.00000012e-06,
       9.99999997e-07, 1.00000012e-06, 1.00000012e-06, 9.99999997e-07]), flows=array([3000.        ,    0.        , 2173.31384199,  826.68615801,
          0.        ,    0.        , 2173.31384199,    0.        ,
        826.68615801,    0.        ,    0.        , 3000.        ,
          0.        , 3000.        , 3000.        ,    0.        ]), origins=array([0], dtype=uint32), destinations=array([1], dtype=uint32), origin_flows=None, destination_flows=array([[3000.        ,    0.        , 2173.31384199,  826.68615801,
           0.        ,    0.        , 2173.31384199,    0.        ,
         826.68615801,    0.        ,    0.        , 3000.        ,
           0.        , 3000.        , 3000.        ,    0.        ]]), skim=array([[0.06046158]]), g

### First let's solve a toy problem

In [63]:
# load a silly network
g = get_toy_network('cascetta')
labeled_g = relabel_graph(g)
show_network(labeled_g, euclidean=True)

In [58]:
g.nodes.data()

NodeDataView({1: {'x_coord': 2, 'y_coord': 1.4142135623730951, 'ctrl_type': 'none', 'node_id': 0}, 2: {'x_coord': 3.414213562373095, 'y_coord': 2.8284271247461903, 'ctrl_type': 'none', 'node_id': 1}, 3: {'x_coord': 3.414213562373095, 'y_coord': 0, 'ctrl_type': 'none', 'node_id': 2}, 4: {'x_coord': 4.82842712474619, 'y_coord': 1.4142135623730951, 'ctrl_type': 'none', 'node_id': 3}})

In [68]:
# we've picked some random centroids that are close to nodes 1 and 4 in the original graph
centroid_x = np.array([1, 7])
centroid_y = np.array([1, 1])

# add these centroids as "centroid" nodes (against which O-D flows will be specified)
# also automatically adds connectors (with capacities, etc.) from the centroids to the nearest nodes on the node-link graph
fg = add_centroids(g, centroid_x, centroid_y, euclidean=True, method="turn") # on the unlabeled graph
fg = relabel_graph(fg)  # adding link and node ids, connectors and centroids
show_network(fg, euclidean=True)

C:\Users\nicholas.padon\AppData\Local\Programs\Python\Python311\Lib\site-packages\dyntapy\demand_data.py:829: UserWarning: The `euclidean_dist_vec` function has been renamed `euclidean`. Calling `euclidean_dist_vec` will raise an error in a future release.
  dists = euclidean_dist_vec(
C:\Users\nicholas.padon\AppData\Local\Programs\Python\Python311\Lib\site-packages\dyntapy\demand_data.py:829: UserWarning: The `euclidean_dist_vec` function has been renamed `euclidean`. Calling `euclidean_dist_vec` will raise an error in a future release.
  dists = euclidean_dist_vec(


In [42]:
# and for the demand (we want 3000 demand going from centroid 0->1 (which corresponds to node 1->4 on the node-link graph)
od_matrix = np.zeros(len(centroid_x)*len(centroid_y)).reshape((len(centroid_x), len(centroid_y)))
od_matrix[0, 1] = 3000
od_graph = od_graph_from_matrix(od_matrix, centroid_x, centroid_y)
# nx.draw(od_graph,with_labels=True)

In [43]:
od_graph.nodes.data()

NodeDataView({0: {'x_coord': 1, 'y_coord': 1}, 1: {'x_coord': 7, 'y_coord': 1}})

In [44]:
od_graph.edges.data()

OutEdgeDataView([(0, 1, {'flow': 3000.0})])

In [45]:
%%time
sa = StaticAssignment(fg, od_graph)

init passed successfully
CPU times: total: 0 ns
Wall time: 3.66 ms


In [46]:
%%time
result = sa.run(method='dial_b') # uses numba, so first run needs compilation time

solution found, Dial B in iteration 1
CPU times: total: 0 ns
Wall time: 649 µs


In [47]:
result

StaticResult(link_costs=array([5.41853649e-06, 1.10536212e-05, 2.50000004e-02, 4.39843757e-02,
       5.41196096e-06, 2.35702265e-02, 2.59248024e-02, 2.50000004e-02,
       2.72704162e-01, 2.46779622e-01, 2.50000004e-02, 1.66666675e-02,
       2.50000004e-02, 1.10670513e-05]), flows=array([3000.        ,    0.        ,    0.        , 3000.        ,
          0.        ,    0.        , 1409.40032808,    0.        ,
       1590.59967192, 1409.40032808,    0.        ,    0.        ,
          0.        , 3000.        ]), origins=array([0], dtype=uint32), destinations=array([1], dtype=uint32), origin_flows=None, destination_flows=array([[3000.        ,    0.        ,    0.        , 3000.        ,
           0.        ,    0.        , 1409.40032808,    0.        ,
        1590.59967192, 1409.40032808,    0.        ,    0.        ,
           0.        , 3000.        ]]), skim=array([[0.05094129]]), gap_definition='epsilon=0.0001 converged destination - bushes divided by total origins, becom

In [40]:
[(f, some_edge) for f, some_edge in zip(result.flows, fg.edges(data=True)) if 
 #    some_edge[2].get('connector',False)==False and 
 f > 0]

[(3000.0,
  (0,
   6,
   {'connector': True,
    'length': 0.1,
    'free_speed': 200.0,
    'lanes': 10,
    'capacity': 10000.0,
    'link_type': 1,
    'link_id': 0,
    'from_node_id': 0,
    'to_node_id': 6})),
 (2173.3139814415235,
  (2,
   3,
   {'length': 2.0,
    'capacity': 2000,
    'free_speed': 80,
    'lanes': 1,
    'link_id': 2,
    'from_node_id': 2,
    'to_node_id': 3})),
 (826.6860185584768,
  (2,
   4,
   {'length': 2.0,
    'capacity': 2000,
    'free_speed': 80,
    'lanes': 1,
    'link_id': 3,
    'from_node_id': 2,
    'to_node_id': 4})),
 (2173.3139814415235,
  (3,
   5,
   {'length': 2.0,
    'capacity': 2000,
    'free_speed': 80,
    'lanes': 1,
    'link_id': 6,
    'from_node_id': 3,
    'to_node_id': 5})),
 (826.6860185584768,
  (4,
   5,
   {'length': 2.0,
    'capacity': 500,
    'free_speed': 120,
    'lanes': 1,
    'link_id': 8,
    'from_node_id': 4,
    'to_node_id': 5})),
 (3000.0000000000005,
  (5,
   7,
   {'connector': True,
    'length': 0.0